#### IMPORTS

In [1]:
#Importa os módulos usados

import numpy as np # type: ignore
import matplotlib.pyplot as plt # type: ignore
import copy


# Define um tipo de dado similar ao Pascal "record" or C "struct"

class Struct:
    pass

#### Solução Inicial

In [2]:
# heuristica

def heuristica_insercao_mais_proxima(clientes, num_pontos_acesso):
    # Escolha aleatória do primeiro ponto de acesso
    pa_inicial = np.random.randint(num_pontos_acesso)
    solucao = np.zeros((num_pontos_acesso, len(clientes)))  # Matriz X
    ativacao_pa = np.zeros(num_pontos_acesso)  # Vetor Y

    # Calcular distância euclidiana entre clientes e pontos de acesso
    distancias = np.zeros((len(clientes), num_pontos_acesso))
    for i, cliente in enumerate(clientes):
        for j in range(num_pontos_acesso):
            distancias[i, j] = np.sqrt((cliente.x - j) ** 2 + (cliente.y - j) ** 2)

    # Atribuir cada cliente ao ponto de acesso mais próximo
    for j in range(len(clientes)):
        distancias_cliente = distancias[j]
        pa_mais_proximo = np.argmin(distancias_cliente)
        solucao[pa_mais_proximo, j] = 1  # Define o cliente j para o PA mais próximo
        ativacao_pa[pa_mais_proximo] = 1  # Ativa o PA mais próximo

    return solucao, ativacao_pa

# Solução inicial
def sol_inicial(probdata, apply_heur):
    
    if apply_heur:
        x = Struct()
        y = Struct()
        x.solution, y.solution = heuristica_insercao_mais_proxima(probdata.nCL, probdata.nPA)
    
    else:
        x = Struct()
        x.solution = np.random.randint(0,1,size=(probdata.nCL, probdata.nPA))
        y = Struct()
        y.solution = np.random.randint(0,1,size=(1, probdata.nPA))

        # IMPLEMENTAR METAHEURÍSTICA CONSTRUTIVA AQUI
    
    return x, y

#### Funções UTIL para definição de dados

In [5]:
class Cliente:
    def __init__(self, x, y, consumo_banda):
        self.x = x
        self.y = y
        self.consumo_banda = consumo_banda

    def __repr__(self):
        return f"Cliente(x={self.x}, y={self.y}, consumo_banda={self.consumo_banda})"

# Função para ler os clientes do arquivo
def ler_clientes(nome_arquivo):
    clientes = []
    with open(nome_arquivo, 'r') as arquivo:
        linhas = arquivo.readlines()
        for linha in linhas:
            x, y, consumo_banda = map(float, linha.strip().split(','))
            cliente = Cliente(x, y, consumo_banda)
            clientes.append(cliente)
    return clientes

# Uso da função para ler os clientes do arquivo
clientes = ler_clientes('data/clientes.csv')

# Exemplo de uso: imprimir os clientes lidos
# for cliente in clientes:
#    print(cliente)

In [7]:
def clientProcessing(clientList, nCL=495, nPA=81**2):

    cons = np.zeros(nCL)
    dist = np.zeros((nCL, nPA))
    exp = np.zeros((nCL, nPA))

    cl_index = 0
    for cliente in clientList:
        cons[cl_index] = cliente.consumo_banda

        for PA_index in range(nPA):
            PA_x = (PA_index % 81) * 5
            PA_y = (PA_index // 81) * 5
            
            dist[cl_index, PA_index] = np.sqrt((cliente.x - PA_x)**2 + (cliente.y - PA_y)**2)
            #if PA_index % 81 == 80 & PA_index // 81 == 80:
            #    print(PA_x, end="-")
            #    print(PA_y, end=" to ")
            #    print(cliente.x, end="-")
            #    print(cliente.y, end=": ")
            #    print(dist[cl_index, PA_index])
            exp[cl_index, PA_index] = 1/dist[cl_index, PA_index]

        cl_index += 1

    return cons, dist, exp

C, D, E = clientProcessing(clientes)

400-400 to 36.524-141.41: 446.0757678646084
400-400 to 68.055-53.432: 479.8925521916338
400-400 to 1.2006-65.695: 520.3852365943522
400-400 to 171.29-68.62: 402.64248223455013
400-400 to 90.213-113.73: 421.803862321103
400-400 to 21.83-131.84: 463.59716834769387
400-400 to 86.03-72.527: 453.6691819255525
400-400 to 7.8241-64.146: 516.3330764698403
400-400 to 109.12-126.68: 399.14282756928003
400-400 to 70.927-116.16: 434.57356676286696
400-400 to 82.1-104.41: 434.0896890044729
400-400 to 98.092-243.31: 340.1473159735352
400-400 to 122.69-81.585: 422.24275994385033
400-400 to 39.511-97.064: 470.87422866090265
400-400 to 6.6475-75.823: 509.7223916851702
400-400 to 74.805-110.82: 435.17451720545404
400-400 to 38.854-53.75: 500.315396341148
400-400 to 101.25-130.39: 402.41907832507144
400-400 to 25.468-171.46: 438.7536331747009
400-400 to 122.93-84.264: 420.0678595131982
400-400 to 95.32-76.584: 444.32849498541054
400-400 to 24.422-51.827: 512.1359897654138
400-400 to 64.138-56.321: 480.53

#### Função definição de dados para o problema

In [ ]:
def probdef(nPA=81**2, nCL=495):
    
    probdata = Struct()

    probdata.CL_cons = 0 # importar consumo dos clientes do .csv # TODO

    probdata.PA_cap = 54 # em Mbps
    probdata.PA_raio = 84 # em metros
    probdata.CL_min_p = 0.05 # porcentagem minima de clientes
    probdata.exp_coef = 1 # coeficiente de exposição
    probdata.falloff = 1 # fator de decaimento

    # CALCULAR DEPOIS
    probdata.dist_CL_PA = np.zeros((nCL, nPA)) # distância entre clientes e PAs # TODO
    probdata.exp_CL_PA = np.zeros((nCL, nPA)) # exposição dos clientes aos PAs # TODO

    probdata.nPA = nPA
    probdata.nCL = nCL
    probdata.nPA_max = 30

    return probdata

#### Função penalidades

In [5]:
# Função de penalidades para todas as restrições
def penalties(x, y, probdata):

    # percentual mínimo de clientes
    pen_CLmin = probdata.nCL * probdata.CL_min_p - np.sum(x.solution) 
    pen_CLmin = np.sum(np.where(pen_CLmin <= 0, 0, pen_CLmin)**2)

    # limite de consumo dos PAs
    pen_PAcap = np.zeros(probdata.nPA)
    for i in range(probdata.nPA):
        pen_PAcap[i] = np.dot(np.array(probdata.CL_cons), np.array(x.solution[:[i]])) - y.solution[i] * probdata.PA_cap
        pen_PAcap[i] = np.sum(np.where(pen_PAcap[i] <= 0, 0, pen_PAcap[i])**2)
    
    # limite de distância entre PAs e clientes
    pen_dist = np.zeros((probdata.nCL, probdata.nPA))
    for i in range(probdata.nPA):
        for j in probdata.nCL:
            pen_dist[j,i] = probdata.dist_CL_PA[j,i] * x.solution[j,i] - y.solution[i] * probdata.PA_raio
            pen_dist[j,i] = np.sum(np.where(pen_dist[j,i] <= 0, 0, pen_dist[j,i])**2)

    # pelo menos 5% de exposição à rede
    pen_CLmin = np.zeros(probdata.nCL)
    for j in range(probdata.nCL):
        pen_CLmin[j] = 0.05 * 1 - np.dot(np.array(probdata.exp_CL_PA[j]), np.array(y.solution))
        pen_CLmin[j] = np.sum(np.where(pen_CLmin[j] <= 0, 0, pen_CLmin[j])**2)
    
    # no máximo um PA por cliente
    pen_PAperCL = np.zeros(probdata.nCL)
    for j in range(probdata.nCL):
        pen_PAperCL[j] = sum(x.solution[j]) - 1
        pen_PAperCL[j] = np.sum(np.where(pen_PAperCL[j] <= 0, 0, pen_PAperCL[j])**2)

    # número máximo de PAs
    pen_PAmax = sum(y.solution) - probdata.nPA_max

    # return all multiplied by U
    return 100 * (pen_CLmin + sum(pen_PAcap) + sum(pen_dist) + sum(pen_CLmin) + sum(pen_PAperCL) + pen_PAmax)

#### Funções Objetivo

In [6]:
# Função objetivo 1: Minimizar número de PAs ativos
def fobj_minPA(x, y, probdata):

    sol = np.transpose(np.array(y.solution))

    y.fitness = np.sum(sol)
    y.penalidade = penalties(x, y, probdata)
    y.fitness_penalizado = y.fitness + y.penalidade

    return y


# Função objetivo 2: Minimizar distância cumulativa de clientes e PAs
def fobj_mindist(x, y, probdata):

    sol = x.solution

    fit_matrix = np.multiply(sol, probdata.dist_CL_PA)
    x.fitness = sum(fit_matrix)
    x.penalidade = penalties(x, y, probdata)
    x.fitness_penalizado = x.fitness + x.penalidade

    return x

#### Funções UTIL para VNS

In [8]:
# Shake implementation // TODO: MUDAR HEURISTICAS
def shake(x,k,probdata):
    
    y = copy.deepcopy(x)
    r = np.random.permutation(probdata.n)       
    
    if k == 1:             # apply not operator in one random position
        y.solution[r[0]] = not(y.solution[r[0]])
        
    elif k == 2:           # apply not operator in two random positions        
        y.solution[r[0]] = not(y.solution[r[0]])
        y.solution[r[1]] = not(y.solution[r[1]])
        
    elif k == 3:           # apply not operator in three random positions
        y.solution[r[0]] = not(y.solution[r[0]])
        y.solution[r[1]] = not(y.solution[r[1]])
        y.solution[r[2]] = not(y.solution[r[2]])        
    
    return y

# VND implemantation
def VND(x, k_max):

  k = 1
  while k <= k_max:
    # Encontra o melhor vizinho na vizinhança atual
    x_best = BestImprovement(x)
    # Muda a vizinhança
    x, k = neighborhoodChange(x, x_best, k)

  return x

def BestImprovement(x):
    x_best = x.copy()  # Cópia da solução inicial para guardar o melhor
    while True:
        improved = False
        for y in generate_neighborhood(x):
            if objective_function(y) < objective_function(x):
                x = y.copy()  # Atualiza a solução com a melhoria
                improved = True
                break  # Sai do loop interno se encontrar melhoria
        if not improved:
            break  # Sai do loop externo se não encontrar melhoria
    return x_best

def generate_neighborhood(x, k, probdata):
    # Gera uma vizinhança de soluções a partir da solução atual para um determinado valor de k.

    neighborhood = []

    # Gere os vizinhos aplicando shake com o mesmo valor de k
    for _ in range(probdata.num_neighbors):  # Determine o número de vizinhos a serem gerados
        neighbor = shake(x, k, probdata)
        neighborhood.append(neighbor)

    return neighborhood

# NeighborhoodChange implementation
def neighborhoodChange(x, xlinha, k):
    
    if xlinha.fitness_penalizado < x.fitness_penalizado:
        x = copy.deepcopy(xlinha)
        k  = 1
    else:
        k += 1
        
    return x, k

#### Refinamento(Busca Local)

In [ ]:
def avaliar_solucao(solucao_x, solucao_y, clientes):
    num_pontos_acesso, num_clientes = solucao_x.shape
    custo_total = 0

    # Calcular o custo total da solução
    for i in range(num_pontos_acesso):
        for j in range(num_clientes):
            custo_total += solucao_x[i, j] * clientes[j].consumo_banda * solucao_y[i]

    return custo_total

def busca_local(solucao_x, solucao_y, clientes, num_pontos_acesso, max_iter):
    melhor_solucao_x = np.copy(solucao_x)
    melhor_solucao_y = np.copy(solucao_y)
    melhor_avaliacao = avaliar_solucao(melhor_solucao_x, melhor_solucao_y)

    for _ in range(max_iter):
        # Faça uma perturbação na solução atual (por exemplo, trocando aleatoriamente alguns clientes entre pontos de acesso) chamar shake

        # Refine a solução perturbada usando uma heurística local (por exemplo, troca de clientes entre pontos de acesso para melhorar a solução)

        # Avalie a nova solução
        avaliacao = avaliar_solucao(solucao_x, solucao_y)

        # Atualize a melhor solução se a nova solução for melhor
        if avaliacao < melhor_avaliacao:
            melhor_solucao_x = np.copy(solucao_x)
            melhor_solucao_y = np.copy(solucao_y)
            melhor_avaliacao = avaliacao

    return melhor_solucao_x, melhor_solucao_y

#### VNS IMPLEMENTATION

In [ ]:
from time import time

def GVNS(x, l_max, k_max, t_max):
  ##  r_max: 
  ##  k_max: 
  ##  t_max: 
  start_time = time()
  current_solution = x
  best_solution = current_solution.copy()

  while True:
    k = 1
    while k <= k_max:
      # Gera uma solução perturbada
      perturbed_solution = shake(current_solution, k)

      # Aplica VND para refinar a solução perturbada
      improved_solution = VND(perturbed_solution.copy(), l_max)

      # Muda a vizinhança
      best_solution, k = neighborhoodChange(best_solution, improved_solution, k)

      # Verifica critério de parada por tempo
      if time() - start_time > t_max:
        return best_solution

    # Verifica critério de parada por iterações da vizinhança
    if k > k_max:
      break

  return best_solution